In [1]:
from dotenv import load_dotenv
load_dotenv()

from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SimpleNodeParser
import os
import weaviate
from weaviate.classes.init import Auth

wcd_url = os.environ["WCD_URL"]
wcd_api_key = os.environ["WCD_API_KEY"]

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,                                    
    auth_credentials=Auth.api_key(wcd_api_key),
)

In [2]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

embed_model = OpenAIEmbedding(model="text-embedding-3-large")

Settings.embed_model = embed_model

embeddings = embed_model.get_text_embedding(
    "OpenAI new Embeddings models is great."
)

print(embeddings[:5])

print("\n")

print(len(embeddings))


[-0.007724063005298376, 0.039427097886800766, -0.017061689868569374, -0.02000819891691208, 0.024540210142731667]


3072


In [3]:
# Ingest the docs in the data folder

docs = SimpleDirectoryReader('./data').load_data()

parser = SimpleNodeParser()
nodes = parser.get_nodes_from_documents(docs)


In [4]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.weaviate import WeaviateVectorStore


# construct vector store
vector_store = WeaviateVectorStore(weaviate_client = client, index_name="James")

# setting up the storage for the embeddings
storage_context = StorageContext.from_defaults(vector_store = vector_store)

# set up the index
index = VectorStoreIndex(nodes, storage_context = storage_context)


In [5]:
query_engine = index.as_query_engine()
response = query_engine.query("How can I maximize my charge?")
print(response)


To maximize your charge, it is recommended to avoid letting the Battery capacity drop too low, refer to the vehicle touchscreen or mobile app for charging limits, ensure the Wall Connectors are configured for a 48A charge current, confirm the charge port light blinks green when charging, and follow proper procedures when unlatching the charge cable.
